In [26]:
## let's start
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [27]:
# read data
with open ('input_abcd.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [28]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  244720


In [29]:
print(text[:10])

abcd abcd 


In [30]:
# unique characters
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

 abcd
5


In [31]:
# create character to integer mapping
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # input string and output integers
decode = lambda l: ''.join([itos[i] for i in l]) # input integers and outpu string

print(encode("bda "))
print(decode(encode("bda ")))

[2, 4, 1, 0]
bda 


In [32]:
#question for future 1: large vocab/small sequence how is the data read to split into tokens

In [33]:
#encode entire text and store into torch tensor
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # first 100 characters - encoded

torch.Size([244720]) torch.int64
tensor([1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
        1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
        1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
       

In [34]:
# split data into training and validation set
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [35]:
block_size=8
train_data[:block_size+1]

tensor([1, 2, 3, 4, 0, 1, 2, 3, 4])

In [36]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 2
when input is tensor([1, 2]) the target: 3
when input is tensor([1, 2, 3]) the target: 4
when input is tensor([1, 2, 3, 4]) the target: 0
when input is tensor([1, 2, 3, 4, 0]) the target: 1
when input is tensor([1, 2, 3, 4, 0, 1]) the target: 2
when input is tensor([1, 2, 3, 4, 0, 1, 2]) the target: 3
when input is tensor([1, 2, 3, 4, 0, 1, 2, 3]) the target: 4


In [199]:
## remove later


import torch
import random
import numpy as np

seed = 42

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [200]:
import torch
import torch.nn as nn
from torch.nn import functional as F


#hyperparameters
batch_size=4 # independent sequences in parallel
block_size=8 # context length
max_iters=1000
eval_interval=100
learning_rate=3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'
eval_iters=20
n_embd=4
n_head=2
n_layer=6
dropout=0.1

#--------------------
#torch.manual_seed(1337)


def get_batch(split):
    #generate a small batch of inputs x and targets y
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

xb, yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

# for i in range(batch_size):  #batch dimension - rows
#     for t in range(block_size): #time dimension - column
#         context=xb[i,:t+1]
#         target=yb[i,t]
#         print(f"when input is {context} output is: {target}")


inputs:
torch.Size([4, 8])
tensor([[3, 4, 0, 1, 2, 3, 4, 0],
        [3, 4, 0, 1, 2, 3, 4, 0],
        [2, 3, 4, 0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4, 0, 1, 2]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[4, 0, 1, 2, 3, 4, 0, 1],
        [4, 0, 1, 2, 3, 4, 0, 1],
        [3, 4, 0, 1, 2, 3, 4, 0],
        [1, 2, 3, 4, 0, 1, 2, 3]], device='cuda:0')


In [201]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            X,Y=X.to(device),Y.to(device)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out


In [202]:


class Head(nn.Module):
    """One head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        B,T,C = x.shape
        k=self.key(x) # (B,T,C head_size)
        q=self.query(x) # (B,T,C head_size)
        # compute attention scores - affinities
        wei = q @ k.transpose(-2,-1) *C**-0.5 # (B,T,16) * (B,16,T) --> (B,T,T)
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) #(B,T,T)
        wei=F.softmax(wei,dim=-1) #(B,T,T)
        wei=self.dropout(wei)
        # perform weighted aggregation of the values
        v=self.value(x) # (B,T,C)
        out=wei @ v #(B,T,T) @ (B,T,C) -> (B,T,C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self attention in parallel"""

    def __init__(self, num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [203]:
class FeedForward(nn.Module):
    """a simple linear layer followed by non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd), #projection layer, added in multi head attention
            nn.Dropout(dropout)
        )
    
    def forward(self,x):
        return self.net(x)
    
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd,n_head):
        #n_embd - number of embeddings, n_head = number of heads
        super().__init__()
        head_size = n_embd//n_head
        self.sa_heads=MultiHeadAttention(n_head,head_size) # heads of dimensional self-attention
        self.ffwd=FeedForward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)

    def forward(self,x):
        x=x + self.sa_heads(self.ln1(x)) # apply 4 heads of 8 dimensional self attention (B,T,C)
        x=x + self.ffwd(self.ln2(x)) # (B, T, C)  
        return x

In [204]:
# backup

# class BigramLanguageModel(nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         #each token directly reads off the logits for the next token from a lookup table
#         self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
#         self.position_embedding_table=nn.Embedding(block_size,n_embd)
#         #self.sa_head=Head(n_embd)
#         self.sa_heads=MultiHeadAttention(4,n_embd//4) # 4 heads of 8 dimensional self-attention
#         self.ffwd=FeedForward(n_embd)
#         self.lm_head=nn.Linear(n_embd,vocab_size)
    
#     def forward(self,idx,targets=None):
#         B,T = idx.shape
#         #idx and targets are both B,T tensor of integers
#         tok_emb= self.token_embedding_table(idx) #(B,T,C)
#         pos_emb=self.position_embedding_table(torch.arange(T,device=device)) #(T,C)
#         x = tok_emb + pos_emb #(B,T,C)
#         #x=self.sa_head(x) # apply one head of self attention (B,T,C)
#         x=self.sa_heads(x) # apply 4 heads of 8 dimensional self attention (B,T,C)
#         x=self.ffwd(x) # (B, T, C)
#         logits=self.lm_head(x) #(B,T,vocab_size)   

#         if targets is None:
#             loss = None
#         else: 
#             B,T,C=logits.shape
#             logits=logits.view(B*T,C)
#             targets=targets.view(B*T)
#             loss=F.cross_entropy(logits,targets)

#         return logits,loss

In [205]:





class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        #print(self.token_embedding_table.weight)  # abcd 5*4
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        #print(self.position_embedding_table.weight)  # abcd 8*4
        # self.blocks = nn.Sequential(
        #     Block(n_embd,n_head=4),
        #     Block(n_embd,n_head=4),
        #     Block(n_embd,n_head=4),
        #     nn.LayerNorm(n_embd)
        # )
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        #print(self.blocks)  # abcd 
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        #print(self.ln_f.weight)  # abcd  
        self.lm_head=nn.Linear(n_embd,vocab_size)
        #print(f"weights - {self.lm_head.weight}") # abcd 4*5
        #print(f"bias - {self.lm_head.bias}") # abcd
    
    def forward(self,idx,targets=None):
        B,T = idx.shape
        #idx and targets are both B,T tensor of integers
        #print("Forward") # abcd
        tok_emb= self.token_embedding_table(idx) #(B,T,C)
        #print(tok_emb[0,:,:]) # abcd
        pos_emb=self.position_embedding_table(torch.arange(T,device=device)) #(T,C)
        #print(pos_emb) # abcd
        x = tok_emb + pos_emb #(B,T,C)
        #print(x[0,:,:]) # abcd
        #x=self.sa_head(x) # apply one head of self attention (B,T,C)
        x=self.blocks(x) #(B,T,C)
        x=self.ln_f(x) #(B,T,C)
        logits=self.lm_head(x) #(B,T,vocab_size)
        #print(logits) # abcd  

        if targets is None:
            loss = None
        else: 
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to last block_size tokens
            idx_cond=idx[:,-block_size:]
            # get the predictions
            logits,loss =self(idx_cond)
            # focus only on the last time step
            logits=logits[:,-1,:] #becomes B,C
            # apply softmax to get the probabilities
            probs=F.softmax(logits,dim=-1) # B,C
            # sample from the distribution
            idx_next=torch.multinomial(probs,num_samples=1) # B,1
            # append sample index to the running sequence
            idx=torch.cat((idx, idx_next),dim=1) #(B,T+1)
        return idx


model=BigramLanguageModel()
model=model.to(device)
#out=m(xb,yb)
# logits,loss=model(xb,yb)
# print(logits.shape)
# print(loss)

idx=torch.zeros((1,1),dtype=torch.long)
#print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

In [206]:
# create a pytorch optimizer
optimizer=torch.optim.AdamW(model.parameters(),lr=1e-3)

In [207]:
def collect_params(model):
    params = {}

    for name, param in model.named_parameters():
        params[name] = param.detach().cpu().clone()

    return params

def print_params(params, title, max_elems=5):
    print(f"\n===== {title} =====")
    for name, tensor in params.items():
        print(f"\n{name}")
        print(f"shape = {tuple(tensor.shape)}")
        print(tensor)


init_params = collect_params(model)
print_params(init_params, "INITIALIZATION (before forward)")


===== INITIALIZATION (before forward) =====

token_embedding_table.weight
shape = (5, 4)
tensor([[ 0.6784, -1.2345, -0.0431, -1.6047],
        [ 0.3559, -0.6866, -0.4934,  0.2415],
        [-1.1109,  0.0915, -2.3169, -0.2168],
        [-0.3097, -0.3957,  0.8034, -0.6216],
        [-0.5920, -0.0631, -0.8286,  0.3309]])

position_embedding_table.weight
shape = (8, 4)
tensor([[ 0.0349,  0.3211,  1.5736, -0.8455],
        [ 1.3123,  0.6872, -1.0892, -0.3553],
        [-1.4181,  0.8963,  0.0499,  2.2667],
        [ 1.1790, -0.4345, -1.3864, -1.2862],
        [-0.8371, -0.9224,  1.8113,  0.1606],
        [ 0.3672,  0.1754,  1.3852, -0.4459],
        [-1.2024,  0.7078, -1.0759,  0.5357],
        [ 1.1754,  0.5612, -0.4527, -0.7718]])

blocks.0.sa_heads.heads.0.key.weight
shape = (2, 4)
tensor([[ 0.4811, -0.4126, -0.4959, -0.3912],
        [-0.3363,  0.2025,  0.1790,  0.4155]])

blocks.0.sa_heads.heads.0.query.weight
shape = (2, 4)
tensor([[-0.2582, -0.3409,  0.2653, -0.2021],
        [ 0.303

In [ ]:

# model_cpu = model.eval()
# xb_cpu = xb
# yb_cpu = yb

# with torch.no_grad():
#     # embeddings
#     tok_emb = model_cpu.token_embedding_table(xb_cpu)
#     pos_emb = model_cpu.position_embedding_table(
#         torch.arange(8,device=device))
#     x1 = tok_emb + pos_emb

#     # transformer blocks
#     x2 = model_cpu.blocks(x1)

#     # FINAL LayerNorm (this is what you want)
#     x_ln = model_cpu.ln_f(x2)

#     # logits
#     logits_cpu = model_cpu.lm_head(x_ln)


In [ ]:
# print(logits_cpu)

tensor([[[-0.6285,  0.3582,  0.4341,  1.5654, -0.0124],
         [ 0.1954, -0.6630,  0.2420,  0.7547,  1.4802],
         [-0.3672,  0.4535,  0.5218,  1.2911,  0.4613],
         [ 0.0346, -0.5020,  0.3491,  1.1050,  1.2939],
         [-0.4190,  0.5429,  0.4794,  1.1957,  0.2875],
         [-0.6757,  0.1729,  0.3796,  1.6698, -0.0717],
         [ 0.2339,  0.1642,  0.1994, -0.1062,  1.1169],
         [-0.1207, -0.5863,  0.3150,  1.3329,  1.0475]],

        [[-0.6285,  0.3582,  0.4341,  1.5654, -0.0124],
         [ 0.1954, -0.6630,  0.2420,  0.7547,  1.4802],
         [-0.3672,  0.4535,  0.5218,  1.2911,  0.4613],
         [ 0.0346, -0.5020,  0.3491,  1.1050,  1.2939],
         [-0.4190,  0.5429,  0.4794,  1.1957,  0.2875],
         [-0.6757,  0.1729,  0.3796,  1.6698, -0.0717],
         [ 0.2339,  0.1642,  0.1994, -0.1062,  1.1169],
         [-0.1207, -0.5863,  0.3150,  1.3329,  1.0475]],

        [[-0.1733, -0.0917,  0.5011,  1.4163,  0.9796],
         [-0.0977, -0.8045,  0.1918,  1.2022

In [208]:
#batch_size=32
for iter in range(max_iters):



    # sample a batch of data
    (xb,yb)=get_batch('train')

    # evaluate the loss
    logits,loss=model(xb,yb)

    #  capture right before FIRST optimizer update
    if iter == 0:
        first_batch=(xb,yb)
        logits_before_opt = logits.detach().cpu().clone()
        

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    #  capture right after FIRST optimizer update
    if iter == 0:
        first_step_params = collect_params(model)
        logits_initial = logits.detach().cpu().clone()

    # evey once in a while evaluate the loss on train and val sets
    if iter % eval_interval==0:
        losses=estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss{losses['val']:.4f}")
        

print(loss.item())

step 0: train loss 2.0087, val loss1.9599
step 100: train loss 1.5707, val loss1.5766
step 200: train loss 1.2793, val loss1.2958
step 300: train loss 0.8688, val loss0.8739
step 400: train loss 0.6385, val loss0.6297
step 500: train loss 0.5086, val loss0.4802
step 600: train loss 0.3719, val loss0.3793
step 700: train loss 0.2736, val loss0.2810
step 800: train loss 0.2007, val loss0.2114
step 900: train loss 0.1412, val loss0.1356
0.1377522349357605


In [209]:

print_params(first_step_params, "AFTER 1st OPTIMIZER STEP")


===== AFTER 1st OPTIMIZER STEP =====

token_embedding_table.weight
shape = (5, 4)
tensor([[ 0.6774, -1.2355, -0.0421, -1.6037],
        [ 0.3549, -0.6876, -0.4924,  0.2425],
        [-1.1099,  0.0905, -2.3159, -0.2178],
        [-0.3087, -0.3947,  0.8024, -0.6206],
        [-0.5930, -0.0621, -0.8295,  0.3299]])

position_embedding_table.weight
shape = (8, 4)
tensor([[ 0.0339,  0.3221,  1.5746, -0.8445],
        [ 1.3133,  0.6862, -1.0882, -0.3563],
        [-1.4190,  0.8953,  0.0489,  2.2677],
        [ 1.1799, -0.4335, -1.3874, -1.2872],
        [-0.8381, -0.9234,  1.8123,  0.1616],
        [ 0.3662,  0.1744,  1.3861, -0.4448],
        [-1.2014,  0.7068, -1.0749,  0.5366],
        [ 1.1764,  0.5602, -0.4537, -0.7708]])

blocks.0.sa_heads.heads.0.key.weight
shape = (2, 4)
tensor([[ 0.4821, -0.4116, -0.4969, -0.3902],
        [-0.3353,  0.2035,  0.1780,  0.4145]])

blocks.0.sa_heads.heads.0.query.weight
shape = (2, 4)
tensor([[-0.2592, -0.3419,  0.2663, -0.2031],
        [ 0.3025, -0.1

In [210]:
final_params = collect_params(model)
logits_final = logits.detach().cpu().clone()
print_params(final_params, "AFTER LAST OPTIMIZER STEP")


===== AFTER LAST OPTIMIZER STEP =====

token_embedding_table.weight
shape = (5, 4)
tensor([[ 1.2184, -1.9322,  0.6959, -1.5293],
        [ 0.4196, -1.1127, -1.0876,  1.1806],
        [-0.9501,  0.3700, -2.5186, -0.5853],
        [-0.6675, -0.0389,  0.9222, -0.6924],
        [-0.6536, -0.2631, -0.6040,  0.5524]])

position_embedding_table.weight
shape = (8, 4)
tensor([[ 0.0548,  0.3112,  1.4023, -0.5806],
        [ 1.2468,  0.5932, -0.8611, -0.4066],
        [-1.1272,  0.8496, -0.0511,  2.0627],
        [ 0.8585, -0.3408, -1.2980, -1.3582],
        [-0.7949, -0.4111,  1.3308, -0.1955],
        [ 0.2491,  0.2504,  1.2860, -0.3715],
        [-1.0116,  0.2084, -0.5759,  0.7111],
        [ 1.2096,  0.4021, -0.4463, -0.6036]])

blocks.0.sa_heads.heads.0.key.weight
shape = (2, 4)
tensor([[ 0.4412, -0.3431, -0.4679, -0.4997],
        [-0.3011,  0.3035,  0.0745,  0.4491]])

blocks.0.sa_heads.heads.0.query.weight
shape = (2, 4)
tensor([[-0.1766, -0.3070,  0.2269, -0.2699],
        [ 0.2770, -0.

In [211]:
print(logits_before_opt[0:8])
print(logits_initial)
print(logits_final)

tensor([[-0.5136,  0.4143,  0.4980,  1.4906,  0.2215],
        [-0.0714, -0.6300,  0.2959,  1.2522,  1.1193],
        [-0.1393,  0.5444,  0.3458,  0.4671,  0.5524],
        [ 0.2334, -0.8939,  0.1138,  0.6010,  1.4805],
        [-0.6890,  0.4714,  0.3167,  1.2914, -0.3050],
        [-0.4228,  0.5005,  0.5003,  1.2855,  0.3269],
        [-0.2089, -0.1537,  0.4843,  1.4844,  0.9307],
        [-0.0242, -0.6088,  0.3070,  1.1882,  1.1971]])
tensor([[-0.5136,  0.4143,  0.4980,  1.4906,  0.2215],
        [-0.0714, -0.6300,  0.2959,  1.2522,  1.1193],
        [-0.1393,  0.5444,  0.3458,  0.4671,  0.5524],
        [ 0.2334, -0.8939,  0.1138,  0.6010,  1.4805],
        [-0.6890,  0.4714,  0.3167,  1.2914, -0.3050],
        [-0.4228,  0.5005,  0.5003,  1.2855,  0.3269],
        [-0.2089, -0.1537,  0.4843,  1.4844,  0.9307],
        [-0.0242, -0.6088,  0.3070,  1.1882,  1.1971],
        [-0.4545,  0.3631,  0.5244,  1.5056,  0.3694],
        [ 0.3703, -1.0020, -0.0116,  0.1999,  1.5831],
        [

In [212]:
print(first_batch)

(tensor([[4, 0, 1, 2, 3, 4, 0, 1],
        [1, 2, 3, 4, 0, 1, 2, 3],
        [1, 2, 3, 4, 0, 1, 2, 3],
        [0, 1, 2, 3, 4, 0, 1, 2]], device='cuda:0'), tensor([[0, 1, 2, 3, 4, 0, 1, 2],
        [2, 3, 4, 0, 1, 2, 3, 4],
        [2, 3, 4, 0, 1, 2, 3, 4],
        [1, 2, 3, 4, 0, 1, 2, 3]], device='cuda:0'))


In [23]:
context=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(model.generate(context,max_new_tokens=100)[0].tolist()))

 d abcd abcd d abcd  abcabcd abcd abcd abcd abcd abcd abcd abcd abcd d acd abcd abbcd acd abcd ad abc


## Self Attention test

In [23]:
torch.manual_seed(1337)
B,T,C=4,8,2 #batch, time, channels
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [24]:
# version 1
xbow=torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        xbow[b,t]=torch.mean(xprev,0)
        
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [25]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1, keepdim=True)
wei
xbow2=wei @ x # wei-> B * T * T  x -> B * T * C, output B * T * C
xbow2[0]
torch.allclose(xbow,xbow2)

False

In [26]:
# version 3
tril = torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)

xbow3=wei @ x
torch.allclose(xbow2,xbow3)


True

In [ ]:
# version 4: self attention
torch.manual_seed(1337)
B,T,C=4,8,32 #batch, time, channels
x=torch.randn(B,T,C)

# single head
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)

k=key(x) # (B,T,16 head_size)
q=query(x) # (B,T,16 head_size)
wei = q @ k.transpose(-2,-1) # (B,T,16) * (B,16,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)

v=value(x)
out=wei @ v

wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [33]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a=a/torch.sum(a,1, keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
c

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])

## Old test

In [ ]:
block_size=8 # context length
batch_size=4 # independent sequences in parallel
ix=torch.randint(len(data)-block_size,(batch_size,))
ix

In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch version: 2.5.1+cu121
CUDA available: True


In [2]:
print(torch.cuda.get_device_name(0))
print(torch.rand(2,2).cuda())

NVIDIA GeForce RTX 4050 Laptop GPU
tensor([[0.6789, 0.4387],
        [0.5568, 0.1342]], device='cuda:0')
